<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/%E7%AC%AC%E5%8D%81%E5%85%AD%E9%80%B1_%E5%9F%BA%E7%A4%8E%E8%87%AA%E7%84%B6%E8%AA%9E%E8%A8%80%E8%99%95%E7%90%86_NLP_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

分析 PTT 電影版
https://www.ptt.cc/bbs/movie/index.html

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
PTT_URL = 'https://www.ptt.cc/bbs/movie/index.html'  # PTT 電影版網址
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'} # 設定 User-Agent，避免被 PTT 擋掉

In [3]:
def get_articles(url, pages=2): # pages=2 代表爬取最近兩頁
    all_articles = []
    for page in range(pages):
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = []
        for r_ent in soup.find_all(class_="r-ent"):
            try:
                link = r_ent.find('a')['href']
                title = r_ent.find('a').text
                date = r_ent.find(class_='date').text.strip()
                author = r_ent.find(class_='author').text
                link = 'https://www.ptt.cc' + link
                articles.append({'title': title, 'link': link, 'date': date, 'author':author})

            except:
                pass

        next_page = soup.find('a', string='‹ 上頁')
        if next_page:
            url = 'https://www.ptt.cc' + next_page['href']
            time.sleep(2) # 避免過度請求
        else:
            break
        all_articles.extend(articles)
    return all_articles

def get_article_content(link):
  try:
      response = requests.get(link, headers=HEADERS)
      soup = BeautifulSoup(response.text, 'html.parser')

      main_content = soup.find(id='main-content')
      #移除掉標籤跟作者資訊
      all_text = main_content.text.split('--')[0]

      pre_texts = main_content.find_all(class_='article-metaline')
      for pre_text in pre_texts:
        all_text = all_text.replace(pre_text.text,'')
      all_text = all_text.replace('\n','').strip()
      return all_text
  except:
      return "無法讀取內文"

articles = get_articles(PTT_URL)
for article in articles:
  article['content'] = get_article_content(article['link'])
  print("標題:"+article['title'])
  print("連結:"+article['link'])

標題:[新聞]《雷霆特攻隊》作者解釋「哨兵」！他的力
連結:https://www.ptt.cc/bbs/movie/M.1747918609.A.E64.html
標題:[新聞] 茱蒂佛斯特嘆年輕演員不在乎電影爛不爛！
連結:https://www.ptt.cc/bbs/movie/M.1747918824.A.03E.html
標題:[討論] 阿湯哥飾演的伊森·韓特有軍人身份嗎？
連結:https://www.ptt.cc/bbs/movie/M.1747922420.A.E4E.html
標題:[普雷] 《青春末世物語》Happyend 
連結:https://www.ptt.cc/bbs/movie/M.1747923837.A.919.html
標題:[史迪奇雷] 星際寶貝史迪奇，就只是史迪奇
連結:https://www.ptt.cc/bbs/movie/M.1747926301.A.904.html
標題:[普雷] MI8根本沒拍出末世感啊
連結:https://www.ptt.cc/bbs/movie/M.1747926331.A.FF2.html
標題:[ 感人雷] 史迪奇真人版很感人
連結:https://www.ptt.cc/bbs/movie/M.1747927566.A.9CD.html
標題:[新聞]《足球尤物》亞曼達拜恩現況曝光！轉戰 On
連結:https://www.ptt.cc/bbs/movie/M.1747929183.A.9E4.html
標題:[新聞] 《神鬼戰士2》主角表示電影正擺脫大男人
連結:https://www.ptt.cc/bbs/movie/M.1747929744.A.A42.html
標題:[討論] 雷霆特攻隊 該怎麼完結?
連結:https://www.ptt.cc/bbs/movie/M.1747931818.A.173.html
標題:[請益] 不可能的任務8 的疑問 有雷
連結:https://www.ptt.cc/bbs/movie/M.1747931982.A.E4D.html
標題:[討論] 大家最喜歡《不可能的任務》電影哪一集？
連結:https://www.ptt.cc/bbs/movie/M.1747933921.A.7C8.html
標題:[討論] 《復仇者聯盟：末日之戰》延期
連結

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [7]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
# 開啟 Google Sheet
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1u7BOjJoRt7kfAXBG0d_WPcSjfLwNDlC94JPLjk4KaK8/edit?usp=sharing')

# 依照分頁名稱取得 worksheet
worksheet = spreadsheet.worksheet('test2')

In [8]:
worksheet

<Worksheet 'test2' id:167508118>

In [9]:
for article in articles:
    row = [article['title'], article['link'], article['date'], article['author'], article['content']]
    worksheet.append_row(row)

print("資料已成功寫入 Google Sheet！")

資料已成功寫入 Google Sheet！


In [10]:
# 讀取所有數據
rows = worksheet.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [11]:
df

,[新聞]《雷霆特攻隊》作者解釋「哨兵」！他的力,https://www.ptt.cc/bbs/movie/M.1747918609.A.E64.html,5/22,XDGEE,看板movie《雷霆特攻隊*》創作者親自解釋「哨兵」角色核心：「他的力量，就是他的問題」先前於 2025 年 5 月 2 日上映的漫威電影宇宙最新力作《雷霆特攻隊*》（Thunderbolts*），首映以來不論在票房，或是口碑評價上都有著十分亮眼的表現，不僅讓多位先前戲份不足的角色有機會嶄露頭角，更引入了一位可能影響未來數部電影發展的關鍵角色——「哨兵」。他不僅是電影中的主角之一，同時也扮演反派角色，象徵角色內心深層的掙扎。「哨兵」最初於2000年由保羅·詹金斯（Paul Jenkins）與繪師傑·李（Jae Lee）所共同創造。儘管擁有壓倒性的力量，哨兵最引人注目的並非他的超能力，而是他那與自己內心拉鋸不斷的矛盾性格。正因如此，《雷霆特攻隊*》這樣圍繞他內心掙扎的劇情，成功為 MCU 注入一股新鮮感，也探討了心理健康與自我認知等深層議題。日前保羅·詹金斯接受外媒《ScreenRant》訪問時，分享了他對這位角色從漫畫走向大銀幕的看法。他表示，《雷霆特攻隊*》準確捕捉了哨兵的核心：「他的力量，就是他的問題。」哨兵代表了「人性二元性」，還隱喻了什麼？詹金斯解釋，這樣的角色設計其實承襲了自古以來敘事傳統中「人性二元性」的母題，如同《化身博士》（Strange Case of Dr Jekyll and Mr Hyde）中的雙重人格。他表示：「我曾寫過《浩克》（The Hulk），我一直覺得他的力量是他的詛咒。哨兵也一樣，這不只是『超人型角色』的另一種詮釋，而是力量帶來困境的故事，觀眾能夠產生共鳴。」當然，其實「哨兵」一角，其實更蘊藏著漫畫產業發展的隱喻。詹金斯指出，這個角色的創作靈感來自漫畫史的演變歷程：「從4、50 年代的隨機創作，到6、70 年代的實驗性思潮，再到 80 年代的艾倫·摩爾（Alan Moore）、90 年代法蘭克·米勒（Frank Miller）將漫畫推向寫實與黑暗，最後進入 2000 年代的反思與復原，哨兵的故事就像這整段歷史的縮影。」https://www.toy-people.com/?p=100016
0,[新聞] 茱蒂佛斯特嘆年輕演員不在乎電影爛不爛！,https://www.ptt.cc/bbs/movie/M.1747918824.A.03...,5/22,XDGEE,看板movie茱蒂佛斯特感嘆很多年輕演員不在乎劇本品質！他們不在乎電影爛不爛，不在乎台詞糟不...
1,[討論] 阿湯哥飾演的伊森·韓特有軍人身份嗎？,https://www.ptt.cc/bbs/movie/M.1747922420.A.E4...,5/22,sonyxeperiau,看板movie防雷線……………今天進戲院看不可能的任務8發現兩個細節1.登上海陸一號直升機之...
2,[普雷] 《青春末世物語》Happyend,https://www.ptt.cc/bbs/movie/M.1747923837.A.91...,5/22,plurrr,看板movie之前上映好像引起小小熱潮，因為太忙沒跟到看完覺得還好 （以下請參考）雷文防雷資...
3,[史迪奇雷] 星際寶貝史迪奇，就只是史迪奇,https://www.ptt.cc/bbs/movie/M.1747926301.A.90...,5/22,darren2586,看板movie 票根防雷 https://i.imgur.com/ZEd9EmH.jpeg...
4,[普雷] MI8根本沒拍出末世感啊,https://www.ptt.cc/bbs/movie/M.1747926331.A.FF...,5/22,mach1210,無法讀取內文
5,[ 感人雷] 史迪奇真人版很感人,https://www.ptt.cc/bbs/movie/M.1747927566.A.9C...,5/22,ChrisAJ,看板movie剛才看完發現板上分享不多而且評價普普想講點不同立場我幾乎從頭哭到尾的程度的感人...
6,[新聞]《足球尤物》亞曼達拜恩現況曝光！轉戰 On,https://www.ptt.cc/bbs/movie/M.1747929183.A.9E...,5/22,moneybuy,看板movie《足球尤物》亞曼達拜恩現況曝光！轉戰 OnlyFans、走樣外型再掀熱議！ht...
7,[新聞] 《神鬼戰士2》主角表示電影正擺脫大男人,https://www.ptt.cc/bbs/movie/M.1747929744.A.A4...,5/23,arsl400,看板movie新聞網址：https://reurl.cc/qG2VdD《神鬼戰士2》保羅麥斯...
8,[討論] 雷霆特攻隊 該怎麼完結?,https://www.ptt.cc/bbs/movie/M.1747931818.A.17...,5/23,arsl400,看板moviehttps://youtu.be/thzHwFh86dM?si=j89aUK3...
9,[請益] 不可能的任務8 的疑問 有雷,https://www.ptt.cc/bbs/movie/M.1747931982.A.E4...,5/23,lienchi,看板movie整片最大的疑惑就是那個核彈井，真的能被AI完全掌控嗎？如果都知道已經被AI控制...


文本情感分析 (Sentiment Analysis)：

    目的： 判斷影評、討論區留言等文本的情感傾向，例如正面、負面或中立。
    應用場景： 了解觀眾對電影的喜好程度、分析電影口碑、監測社群媒體上的討論趨勢。

In [15]:
!pip install transformers
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="uer/roberta-base-finetuned-dianping-chinese") # 使用 bert-base-chinese 模型
result = classifier("這部電影非常好看！")
print(result)

result = classifier("這部電影難看極了")
print(result)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'positive (stars 4 and 5)', 'score': 0.9658221006393433}]
[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.996296226978302}]


In [14]:
result = classifier("這部電影難看極了")
print(result)

[{'label': 'LABEL_1', 'score': 0.742385745048523}]


In [ ]:
result[0]['score']

0.5294545888900757

In [ ]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
import pandas as pd
import json

genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def analyze_social_media_sentiment(row):
    # 將 DataFrame 轉換為適合 prompt 的字串格式
    prompt = f"""
    請分析以下社交媒體上對特定事件的討論，判斷整體風向（正面、負面、中性），並簡要說明你的判斷依據。
    請以 JSON 格式回覆，包含 'sentiment'（整體風向）和 'reasoning'（判斷依據）兩個欄位。

    討論內容：
    {row}
    """

    try:
        response = model.generate_content(prompt)
        response_text = response.text
    except Exception as e:
        print(f"發生錯誤：{e}")

    return response_text # 確保一定會回傳值

In [ ]:
df['內容']

,內容
0,看板movie剛剛看完了小時候老師常常放的影片三個傻瓜藍丘許多人看完都認為他是勇於衝撞體制 ...
1,看板moviehttps://mdpr.jp/cinema/detail/4448144ht...
2,看板movie 日本電影《雪風YUKIKAZE》2025年8月上映 無意間在Yout...
3,看板movie我蠻驚訝的是，迪士尼竟然沒有著手發展以母獅群為主角的衍生作品。一是女性主角是迪...
4,看板movie※ [本文轉錄自 ckshchen 信箱]作者: TtTt4 (夜間飛行) 看...
5,看板movie政治相關版規(新版/寬鬆版):註:下述提到的政治都是非講到台灣現況的政治電影延...
6,看板movie事實上孩之寶在變形金剛的IP上面並沒有賠一屁股，變形金剛：萬獸崛起的票房達到成...
7,看板movie防雷我還滿喜歡這個導演作品中的生活哲學那淡淡的風格讓人沉浸在劇情中喜歡的會很舒...
8,看板movie新聞網址：https://reurl.cc/zpAzkpUDN / 資深演員林...
9,看板movie看到推文有提到G1造型不符合現代審美我以前也這麼覺得直到大黃蜂上映，這部的人物...


In [ ]:
df['sentiment'] = df['內容'].apply(lambda x: analyze_social_media_sentiment(x))

In [ ]:
df['sentiment']

,sentiment
0,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
1,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
2,"```json\n{\n ""sentiment"": ""中性"",\n ""reasoning..."
3,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
4,"```json\n{\n ""sentiment"": ""Neutral"",\n ""reas..."
5,"```json\n{\n ""sentiment"": ""中性"",\n ""reasoning..."
6,"```json\n{\n ""sentiment"": ""Mixed"",\n ""reason..."
7,"```json\n{\n ""sentiment"": ""正面"",\n ""reasoning..."
8,"```json\n{\n ""sentiment"": ""負面"",\n ""reasoning..."
9,"```json\n{\n ""sentiment"": ""Mixed"",\n ""reason..."


In [ ]:
json_string = df['sentiment'][1]
json_string

'```json\n{\n  "sentiment": "正面",\n  "reasoning": "新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：\\n\\n1. **作品人氣:**  漫畫原作銷量突破90萬冊，說明作品本身擁有相當高的人氣基礎，為電影改編奠定了良好的觀眾基礎。\\n2. **演員陣容:**  選用的演員福本莉子和八木勇征都是當下受矚目的演員，他們的參與提升了電影的期待值。演員們對於角色和電影的積極評價也增加了正面氛圍。\\n3. **製作團隊:**  導演松本花奈曾執導多部成功的青春題材電影，經驗豐富，也增加了電影質量的保證。\\n4. **原作者及製作方態度:** 原作者和製作方都對電影改編表示高度期待和信心，並肯定了劇本和演員陣容。\\n5. **劇情簡介:**  “純真愛情故事”、“經典愛情故事”等詞語營造了溫馨浪漫的氛圍，更容易引起正面共鳴。"\n}\n```\n'

In [ ]:
cleaned_json_string = json_string.replace("\\n", "").replace("\n", "").replace("```json", '').replace("```", '')

In [ ]:
cleaned_json_string

'{  "sentiment": "正面",  "reasoning": "新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：1. **作品人氣:**  漫畫原作銷量突破90萬冊，說明作品本身擁有相當高的人氣基礎，為電影改編奠定了良好的觀眾基礎。2. **演員陣容:**  選用的演員福本莉子和八木勇征都是當下受矚目的演員，他們的參與提升了電影的期待值。演員們對於角色和電影的積極評價也增加了正面氛圍。3. **製作團隊:**  導演松本花奈曾執導多部成功的青春題材電影，經驗豐富，也增加了電影質量的保證。4. **原作者及製作方態度:** 原作者和製作方都對電影改編表示高度期待和信心，並肯定了劇本和演員陣容。5. **劇情簡介:**  “純真愛情故事”、“經典愛情故事”等詞語營造了溫馨浪漫的氛圍，更容易引起正面共鳴。"}'

In [ ]:
def get_sentiment_and_reasoning(json_string):
    cleaned_json_string = json_string.replace("\\n", "").replace("\n", "").replace("```json", '').replace("```", '')
    try:
        data = json.loads(cleaned_json_string)
        sentiment = data["sentiment"]
        reasoning = data["reasoning"]
        print(f"情感：{sentiment}")
        print(f"理由：{reasoning}")
    except json.JSONDecodeError as e:
        print(f"JSON 解析錯誤：{e}")
        print(f"原始字串：{json_string}")
        print(f"清理後字串：{cleaned_json_string}")
    return sentiment, reasoning # 確保一定會回傳值

In [ ]:
df['sentiment_json'] = df['sentiment'].apply(lambda x: get_sentiment_and_reasoning(x))

情感：正面
理由：這段文字主要表達了對電影《三個傻瓜》和《春風化雨》的正面評價。文中肯定了主角藍丘的熱情、堅持和為朋友付出的精神，並將其與《春風化雨》中的基汀老師聯繫起來，讚揚他們勇於追求自我、不墨守成規的精神。  作者認為電影的主旨並非單純的衝撞體制，而是鼓勵人們追隨自己的興趣，勇敢選擇自己的人生道路，並對社會的價值觀念提出了質疑，認為不應將選擇專業局限於特定領域。整體語氣積極向上，充滿對理想人生的嚮往和對電影主題的深刻理解，因此判斷整體風向為正面。
情感：正面
理由：新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：1. **作品人氣:**  漫畫原作銷量突破90萬冊，說明作品本身擁有相當高的人氣基礎，為電影改編奠定了良好的觀眾基礎。2. **演員陣容:**  選用的演員福本莉子和八木勇征都是當下受矚目的演員，他們的參與提升了電影的期待值。演員們對於角色和電影的積極評價也增加了正面氛圍。3. **製作團隊:**  導演松本花奈曾執導多部成功的青春題材電影，經驗豐富，也增加了電影質量的保證。4. **原作者及製作方態度:** 原作者和製作方都對電影改編表示高度期待和信心，並肯定了劇本和演員陣容。5. **劇情簡介:**  “純真愛情故事”、“經典愛情故事”等詞語營造了溫馨浪漫的氛圍，更容易引起正面共鳴。
情感：中性
理由：討論主要圍繞電影《雪風YUKIKAZE》的上映消息、劇情背景（二戰日本軍艦）以及台灣與日本歷史關聯（雪風艦在台服役經歷）。  帖文本身並未表達明顯的正面或負面情緒，只是陳述事實並提出關於台灣片商是否引進電影的疑問。  雖然涉及二戰歷史敏感議題，但討論內容並未展現對該議題的負面或批判性情緒，也沒有明顯的讚揚或支持。因此，整體風向判斷為中性。
情感：正面
理由：該討論表達了對迪士尼缺乏以母獅群為主角的衍生作品的遺憾和期待。文中多次使用正面詞彙，例如「帥呆了」、「帥」、「存活關鍵」，並肯定了母獅在電影中的角色和能力，展現出對母獅角色的喜愛和對相關衍生作品的強烈期待。整體語氣積極，表達了對此類作品的渴望，而非批評或負面評價。
情感：Neutral
理由：This text is a set of rules for a movie discussion board.  It outlines acceptable and unacceptable be

In [ ]:
df['sentiment_json']

,sentiment_json
0,"(正面, 這段文字主要表達了對電影《三個傻瓜》和《春風化雨》的正面評價。文中肯定了主角藍丘的..."
1,"(正面, 新聞稿整體呈現正面積極的語氣，主要基於以下幾個方面：1. **作品人氣:** 漫..."
2,"(中性, 討論主要圍繞電影《雪風YUKIKAZE》的上映消息、劇情背景（二戰日本軍艦）以及台..."
3,"(正面, 該討論表達了對迪士尼缺乏以母獅群為主角的衍生作品的遺憾和期待。文中多次使用正面詞彙..."
4,"(Neutral, This text is a set of rules for a mo..."
5,"(中性, 討論內容主要圍繞PTT movie版關於政治討論的新版規的投票結果和規則細節，包含..."
6,"(Mixed, The discussion presents a mixed sentim..."
7,"(正面, 評論者整體表達了對電影的喜愛。例如，他稱讚了導演的生活哲學、淡淡的風格、選角、題材..."
8,"(負面, 雖然新聞報導林照雄先生的過世，並提及女兒林姿佑的悼念之情，整體基調是哀傷和惋惜。網..."
9,"(Mixed, The discussion shows a mixed sentiment..."
